### Import libraries

In [1]:
from sqlalchemy import create_engine, types
from sqlalchemy import text # to be able to pass string
import pandas as pd
from dotenv import dotenv_values



### Connecting to DB


In [2]:
config = dotenv_values()

pg_user = config['POSTGRES_USER']  # align the key label with your .env file !
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

engine = create_engine(url, echo=False)
my_schema = 'team_aa' # update it to your schema

with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {my_schema};'))


### Function for pivot table

In [ ]:
def file_transforming(file, column_name):
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df = df[df["Jahr"] == "Insgesamt"] # Using only 1 column
    df_long = df.melt( # Make a row as a column
        id_vars = ["Jahr"],
        var_name = "year",
        value_name = column_name
    )
    df_long["year"] = pd.to_numeric(df_long["year"], errors="coerce") # Get rid of mistakes with numbers 
    df_long = df_long.dropna(subset=[column_name]) # Drop info with N/A data
    df_final = df_long[df_long["year"] <= 2024] 
    df_final = df_final[["year", column_name]].reset_index(drop=True) # Choosing final data to show 
    return df_final.to_sql(name = column_name, con = engine, schema= my_schema, # Push to DB
                           if_exists= "replace", index = False)



7

In [ ]:
# Files name
electricity_consumptions = "monatszahlen_energie_und_wasserversorgung_abgabe_an_letztverbraucher_stromverbrauch_25-11-2025.csv"
electricity_generation = "monatszahlen_energie_und_wasserversorgung_elektrizitaetserzeugung_brutto_25-11-2025.csv"
renewable_feed_in = "monatszahlen_energie_und_wasserversorgung_stromeinspeisung_erneuerbare_energien_25-11-2025.csv"
fossil_feed_in = "monatszahlen_energie_und_wasserversorgung_stromeinspeisung_fossile_und_andere_energietraeger_25-11-2025.csv"
total_feed_in = "monatszahlen_energie_und_wasserversorgung_stromeinspeisung_insgesamt_25-11-2025.csv"
# Running functions to push 
file_transforming(electricity_consumptions, "consumed_energie_GWh")
file_transforming(electricity_generation, "generated_energie_GWh")
file_transforming(renewable_feed_in, "renewable_feed_in_GWh")
file_transforming(fossil_feed_in, "fossil_feed_in_GWh")
file_transforming(total_feed_in, "total_feed_in_GWh")